### manova of paintype and paincontrol bfl

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt
# from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [3]:
from compare_paincontrol import make_data_paincontrol
from compare_hyperparams import load_feats, feature_importance, load_modZ
from compare_bfl_qsidp import load_qscode, match_question

/gpfs3/well/seymour/users/uhu195/python/extract-py3.7.4-skylake/lib/python3.7/site-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


#### make data

In [74]:
bestIC = 30 #200 #100 #50 #30
qs_ls = None
idp_ls = None

In [75]:
# load bfl
df_bfl_qsidp = make_data_paincontrol(bestIC=bestIC, qs_ls=qs_ls, idp_ls=idp_ls)

df_bfl_qsidp shape=(4558, 33)


In [76]:
df_bfl_qsidp.columns

Index([  'eid',  'bmrc', 'label',       0,       1,       2,       3,       4,
             5,       6,       7,       8,       9,      10,      11,      12,
            13,      14,      15,      16,      17,      18,      19,      20,
            21,      22,      23,      24,      25,      26,      27,      28,
            29],
      dtype='object')

In [77]:
output_ls = ['label', 'eid', 'bmrc']
dff = df_bfl_qsidp.drop(columns=output_ls)

In [78]:
dff.columns = 'f_' + dff.columns.astype(str)

In [79]:
dffl = pd.concat([dff, df_bfl_qsidp['label']], axis=1)

In [80]:
dffl.shape

(4558, 31)

In [81]:
# dffl.columns

In [82]:
from statsmodels.multivariate.manova import MANOVA

In [83]:
formula = "+".join([f'f_{x}' for x in range(bestIC)]) + '~label'
# print(formula)

In [84]:
fit = MANOVA.from_formula(formula, data=dffl)
print(fit.mv_test())

                   Multivariate linear model
                                                               
---------------------------------------------------------------
       Intercept        Value   Num DF   Den DF  F Value Pr > F
---------------------------------------------------------------
          Wilks' lambda 0.9851 30.0000 4527.0000  2.2889 0.0001
         Pillai's trace 0.0149 30.0000 4527.0000  2.2889 0.0001
 Hotelling-Lawley trace 0.0152 30.0000 4527.0000  2.2889 0.0001
    Roy's greatest root 0.0152 30.0000 4527.0000  2.2889 0.0001
---------------------------------------------------------------
                                                               
---------------------------------------------------------------
         label          Value   Num DF   Den DF  F Value Pr > F
---------------------------------------------------------------
          Wilks' lambda 0.9706 30.0000 4527.0000  4.5757 0.0000
         Pillai's trace 0.0294 30.0000 4527.0000  4.5757 0.

- for IC=200, Pillai's trace 0.0810 99.0000 4458.0000  3.9680 0.0000
- for IC=30, Pillai's trace 0.0294 30.0000 4527.0000  4.5757 0.0000

#### paintype

In [99]:
from compare_paintype import make_data_paintype

In [193]:
bestIC = 30 #200 #100 #50 #30
qs_ls = None
idp_ls = None

In [194]:
# load bfl
df_bfl_qsidp = make_data_paintype(bestIC=bestIC, qs_ls=qs_ls, idp_ls=idp_ls)

/well/seymour/users/uhu195/python/extract_npy//labels/label_patients_pain.csv
(2724, 1)
(2724, 6)
df_bfl_qsidp shape=(2280, 36)


In [195]:
# df_bfl_qsidp.columns

In [196]:
# retrain params
X_train, y_train = load_feats(df_bfl_qsidp, bestIC, test_size=0.25, dummies=False,
                              train=False, balance=False, scaler=True)

(2280, 30)


In [197]:
y_train.rename('label', inplace=True)

0                      back pain
1                 osteoarthritis
2                       migraine
3                       migraine
4                 osteoarthritis
                  ...           
2716              osteoarthritis
2720              osteoarthritis
2721                    migraine
2722    irritable bowel syndrome
2723    irritable bowel syndrome
Name: label, Length: 2280, dtype: object

In [198]:
output_ls = [ 'eid', 'irritable bowel syndrome','migraine','back pain','osteoarthritis','bmrc']
dff = df_bfl_qsidp.drop(columns=output_ls)

In [199]:
dff.columns = 'f_' + dff.columns.astype(str)

In [200]:
dffl = pd.concat([dff, y_train], axis=1)

In [201]:
dffl.shape

(2280, 31)

In [202]:
dffl.columns

Index(['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9',
       'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16', 'f_17', 'f_18',
       'f_19', 'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_27',
       'f_28', 'f_29', 'label'],
      dtype='object')

In [203]:
from statsmodels.multivariate.manova import MANOVA

In [204]:
formula = "+".join([f'f_{x}' for x in range(bestIC)]) + '~label'
# print(formula)

In [205]:
fit = MANOVA.from_formula(formula, data=dffl)
print(fit.mv_test())

                   Multivariate linear model
                                                               
---------------------------------------------------------------
       Intercept        Value   Num DF   Den DF  F Value Pr > F
---------------------------------------------------------------
          Wilks' lambda 0.9698 30.0000 2247.0000  2.3317 0.0001
         Pillai's trace 0.0302 30.0000 2247.0000  2.3317 0.0001
 Hotelling-Lawley trace 0.0311 30.0000 2247.0000  2.3317 0.0001
    Roy's greatest root 0.0311 30.0000 2247.0000  2.3317 0.0001
---------------------------------------------------------------
                                                               
---------------------------------------------------------------
         label          Value   Num DF   Den DF  F Value Pr > F
---------------------------------------------------------------
          Wilks' lambda 0.8870 90.0000 6725.2980  3.0552 0.0000
         Pillai's trace 0.1165 90.0000 6747.0000  3.0281 0.

- for IC=200, Pillai's trace 0.3684 348.0000 6489.0000  2.6100 0.0000
- for IC=30, Pillai's trace 0.1165 90.0000 6747.0000  3.0281 0.0000